[SQL INNER JOIN](https://www.w3schools.com/sql/sql_join_inner.asp)

<b> Syntaxe : </b><br>
SELECT column_name(s)
FROM table1 <br>
INNER JOIN table2
ON table1.column_name = table2.column_name;

<b> Objet de la requête : </b><br>
Le mot-clé INNER JOIN sélectionne les enregistrements dont les valeurs correspondent dans les deux tables.

Problème : comment effectuer des fusions ? <br>
<b>Solution : fusions à opérer avec polars</b>

Date : 20-01-24

In [1]:
import duckdb
from skimpy import skim

In [2]:
# Connexion native à DuckDB : pour saisir du SQL sur notebook jupyter
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

In [3]:
# Configurations sur jupysql pour afficher les données en DF
%config SqlMagic.autopandas = True

In [4]:
# Connection jupysql à DuckDB
%sql duckdb:///:default:

Connecting and switching to connection 'duckdb:///:default:'

Requête sql : récupération des données des tables de la BD

In [268]:
%%sql
customers_df <<
SELECT *
FROM sqlite_scan('data/Ventes.db', 'Clients')

Running query in 'duckdb:///:default:'

In [269]:
%%sql
orders_df <<
SELECT *
FROM sqlite_scan('data/Ventes.db', 'Commandes')

Running query in 'duckdb:///:default:'

In [270]:
%%sql
products_df <<
SELECT *
FROM sqlite_scan('data/Ventes.db', 'Produits')

Running query in 'duckdb:///:default:'

Export des tables sous des fichiers .csv

In [271]:
customers_df.to_csv('data/clients.csv')
orders_df.to_csv('data/commandes.csv')
products_df.to_csv('data/produits.csv')

Traitements opérés avec la librairie pandas

In [272]:
import polars as pl

In [273]:
# Suppression de la 1ère colonne (index) des tables exportées
customers_df = pl.read_csv('data/clients.csv').drop('')
orders_df = pl.read_csv('data/commandes.csv').drop('')
products_df = pl.read_csv('data/produits.csv').drop('')

In [274]:
# Exportation à nouveau des fichiers .csv modifiés
customers_df.write_csv('data/clients.csv')
orders_df.write_csv('data/commandes.csv')
products_df.write_csv('data/produits.csv')

In [279]:
%sql SELECT * FROM 'data/clients.csv' LIMIT 3;

Running query in 'duckdb:///:default:'

,column0,column1,column2,column3
0,ClientID,Segment,Prénom,Nom
1,CG-12520,Consumer,Claire,Gute
2,SO-20335,Consumer,Sean,O'Donnell


In [276]:
%sql SELECT * FROM 'data/commandes.csv' LIMIT 3;

Running query in 'duckdb:///:default:'

,Cde,Dt,Livraison,ClientID,ProduitID,Qty,Rabais,ModeLivraison
0,CA-2022-161480,2022-12-25,2022-12-31,RA-19285,FUR-BO-10004015,2,20 %,Standard Class
1,CA-2022-109715,2022-12-09,2022-12-14,AH-10585,OFF-PA-10004965,2,20 %,Standard Class
2,CA-2022-152660,2022-12-04,2022-12-10,CB-12415,OFF-ST-10000532,2,20 %,Standard Class


In [277]:
%sql SELECT * FROM 'data/produits.csv' LIMIT 3;

Running query in 'duckdb:///:default:'

,ProdID,Produit,PrixUnitaire,Coût,Cat,SousCat
0,OFF-PA-10000007,Telephone Message Books with Fax/Mobile Sectio...,3.60,1.9,Office Supplies,Paper
1,OFF-PA-10000019,Xerox 1931,6.48,3.4,Office Supplies,Paper
2,OFF-PA-10000029,Xerox 224,6.48,3.4,Office Supplies,Paper
